In [3]:
import matplotlib.pyplot as plt
import numpy as np
from astroquery.sdss import SDSS
import ipywidgets as widgets
from IPython.display import clear_output
import sys

In [15]:
def search_the_sky(ra_low,ra_high,dec_low,dec_high,step_size=1):
    query = """
SELECT
  ra,dec,
  dered_g as g, dered_r as r,
  err_g, err_r,
  flags
  
FROM
  Star
  
WHERE
  g - r BETWEEN -0.5 AND 2.5
  AND g BETWEEN 14 and 24
  AND ra BETWEEN {} and {}
  AND dec Between {} and {}
"""
    interesting_coords = []
    ra_saved = []
    dec_saved = []
    for i in np.arange(ra_low,ra_high,step_size):
        for j in np.arange(dec_low,dec_high,step_size):
            temp_data = SDSS.query_sql(query.format(i,i+step_size,j,j+step_size), timeout=1000)
            if str(type(temp_data)) != "<class 'NoneType'>":
                
                fig,ax = plt.subplots(1,2,figsize=(20,10))
                ax[0].hexbin(temp_data['g']-temp_data['r'], temp_data['g'],gridsize=150)
                ax[0].set_ylim(24,14)
                ax[0].set_xlabel('g-r color')
                ax[0].set_ylabel('g magnitude')
                ax[0].set_title('Color-Magnitude Hess Plot')
                ax[1].scatter(temp_data['ra'],temp_data['dec'], marker='.', s=0.2, color='black')
                ax[1].set_aspect('equal')
                ax[1].set_xlabel('Right Ascension')
                ax[1].set_ylabel('Declination')
                plt.show()
                try:
                    interesting_coords.append(float(input(
                        "RA: "+str(i)+
                        " Dec: "+str(j)+
                        " Does this look like there's a globular cluster? 1 for yes, 2 for no, 3 to quit: "
                    )))
                    if interesting_coords[-1] == 1:
                        ra_saved.append(i)
                        dec_saved.append(j)
                    elif interesting_coords[-1] == 3:
                        coords_of_interest = np.array([(ra_saved[i],dec_saved[i]) for i in range(0,len(ra_saved))])
                        return coords_of_interest
                except ValueError:
                    coords_of_interest = np.array([(ra_saved[i],dec_saved[i]) for i in range(0,len(ra_saved))])
                    return coords_of_interest
                
                else:
                    pass
            else:
                pass
            clear_output(wait=True)
    coords_of_interest = np.array([(ra_saved[i],dec_saved[i]) for i in range(0,len(ra_saved))])
    return coords_of_interest

In [17]:
coords = search_the_sky(228,230,-90,90,2)

In [18]:
coords

array([[228,  -2],
       [228,   0],
       [228,   2]])

In [11]:
search_the_sky?

In [16]:

from IPython.display import display
button = widgets.Button(description="Click Me!")
button2 = widgets.Button(description="Hmmm!")
output = widgets.Output()

display(button, output)
display(button2, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

Button(description='Hmmm!', style=ButtonStyle())

Output()